# Структуры

**Структура** - это тип данных, который позволяет объединить в себе 0 или больше именованных значений разных типов. Эти значения называются **полями**(field). Пример:

In [1]:
import (
    "fmt"
    "time"
)

// Объявление структуры Employee.
type Employee struct {
    ID        int
    Name      string
    Address   string
    DoB       time.Time
    Position  string
    Salary    int
    ManagerID int
}

// Создание экземпляра Employee.
var dilbert Employee

Обратиться к отдельным полям можно через "." как, например, dilbert.Name. Т.к. dilbert является переменной, её поля тоже являются переменными, т.е. их можно менять также как и получать адрес:

In [2]:
dilbert.Salary -= 1000
fmt.Println(dilbert.Salary)

position := &dilbert.Position
*position += "Senior " + *position
fmt.Sprint(dilbert.Position)

-1000
Senior 


Точку также используют для доступа к полям **даже** в случае работы с указателем на структуру:

In [3]:
var hardWorker *Employee = &dilbert
// Следующее выражение эквивалентно (*hardWorker).Position += " (proactive team player)".
hardWorker.Position += " (proactive team player)"

fmt.Sprint(dilbert.Position)

Senior  (proactive team player)


Поля структуры обычно записывают каждый на своей строке, но последовательные поля одного типа можно объявить вместе:

In [4]:
type Employee struct {
    ID            int
    Name, Address string
    DoB           time.Time
    Position      string
    Salary        int
    ManagerID     int
}

Имя поля структуры **экспортируется**, если оно начинается с большой буквы. Это основной механизм котроля доступа, используемый в Go.

Структура с именем S **не может** создать внутри себя поле типа S: агрегированные значения не могут содержать сами себя(аналогичное ограничение применимо и к массивам). Но структура S может объявить **поле-указатель**, которое является указателем на S. Таким образом можно создавать рекурсивные структуры данных как связный лист или дерево:

In [5]:
type Node struct {
    value int
    next  *Node
}

var head Node = Node{1, nil}
head.next = &Node{2, nil}

fmt.Sprint(head.value, " -> ", head.next.value)

1 -> 2


Нулевое значение структуры формируется из нулевых значений его полей.

**struct{}** называют пустой структурой, ей размер равен 0 и она не хранит какой либо полезной информации.

## Литерал структуры

Значение типа структуры можно создать с помощью **литерала структуры**, который определяет значения его полей. Например:

In [6]:
type Point struct {X, Y int}

p := Point{1, 2}
fmt.Sprint(p.X, ", ", p.Y)

1, 2


Существует **2 вида** литералов структур: при использовании **первого** вида необходимо указывать значение каждого поля именно в том виде, в котором поля объявлены в структуре. Именно такой литерал был использован в примере выше.  
Гораздо чаще используется **второй** вид литералов, когда поля инициализируются перечислением названия некоторых или всех полей структуры и их значений. Если в таком литерале какое-либо поле пропущено, то для неё устанавливается нулевое значение её типа. Т.к. мы явно указываем название поля, порядок не имеет значения. Пример:

In [7]:
q := Point{Y: -1, X: 8}
fmt.Sprint(q.X, ", ", q.Y)

8, -1


Эти два вида литералов **нельзя** смешивать. Также нельзя с помощью литералов обойти правило экспортирования имён в Go:

package p  
type T struct{ a, b int } // a и b не экспортируются, т.к. начинаются с маленькой буквы.

package q  
import "p"  
var _ = p.T{a: 1, b: 2} // ошибка компиляции: a и b недоступны за пределами пакета p.  
var _ = p.T{1, 2}       // ошибка компиляции: a и b недоступны за пределами пакета p.


Структуры можно передавать в функцию и возвращать из неё:

In [8]:
func Scale(p Point, factor int) Point {
    return Point{p.X * factor, p.Y * factor}
}

p1 := Point{0, 4}
p2 := Scale(p1, 2)
fmt.Sprint(p2)

{0 8}


Большие структуры для эффективности можно передавать и возвращать через указатели, а если нужно изменить значение, то передача через указатель **объязательна**, т.к. в Go передача параметров идёт по значению, а следовательно функция получает копию аргумента, а не ссылку на исходный объект:

In [9]:
func AwardAnnualRaise(e *Employee) {
    e.Salary = e.Salary * 110/100
}

var worker Employee
worker.Salary = 1000

fmt.Println(worker.Salary)
AwardAnnualRaise(&worker)
fmt.Sprint(worker.Salary)

1000
1100


Т.к. часто с структурами работают через указатели можно использовать короткую форму объявления переменной, чтобы инициализировать указатель на структуру:

In [10]:
pp := &Point{3, 1}
fmt.Sprint(*pp)

{3 1}


## Сравнение структур

Если все поля структуры являются сравнимыми, то и сама структура является сравнимой. Оператор == сравнивает соответствующие поля структуры по очереди:

In [11]:
p := Point{1, 0}
q := Point{2, 1}

fmt.Println(p == q)
fmt.Sprint(p == Point{1, 0})

false
true


Сравнимые структуры, также как и другие сравнимые типы, можно использовать в качестве **ключа** словаря:

In [12]:
type Address struct {
    hostname string
    port     int
}

hits := map[Address]int{}
hits[Address{"host.org", 80}]++

## Встраивание структур и анонимные поля

В Go есть так называемый механизм **встраивания структуры**(struct embedding), который позволяет использовать название именованной структуры как **анонимное поле** внутри другой структуры, что в свою очередь позволяет сокращать длиные цепочки обращений к полям типа x.f вместо x.a.b.f. На примере это всё будет намного понятнее.

Представим, что в некой программе двумерного рисования существуют такие фигуры как прямоугольник, эллипс и т.д. Такая программа может объявить, например, следующие 2 типа фигур:

In [13]:
type Circle struct {
    X      int
    Y      int
    Radius int
}

type Wheel struct {
    X      int
    Y      int
    Radius int
    Spokes int // Спицы.
}

Тип Wheel обладает всем свойствами типа Circle и ещё допольнительно количеством спиц. Создать колесо можно так:

In [14]:
var w Wheel
w.X = 0
w.Y = 9
w.Radius = 10
w.Spokes = 30

Т.к. у окружности и круга есть общие поля, можно попытаться выделить их в отдельный тип:

In [15]:
type Point struct {
    X int
    Y int
}

type Circle struct {
    Center Point
    Radius int
}

type Wheel struct {
    Circle Circle
    Spokes int
}

Но в этом случае обращения к полям колеса намного **удлиняются**:

In [16]:
var w Wheel
w.Circle.Center.X = 0
w.Circle.Center.Y = 9
w.Circle.Radius = 10
w.Spokes = 30

Для решения этой проблемы в Go и используется встраивание структур. Go позволяет объявить поле структуры без имени. Такое поле называется **анонимным**. Типом поля должен быть именованный тип или указатель на именованный тип:

In [17]:
type Point struct {
    X int
    Y int
}

type Circle struct {
    Point
    Radius int
}

type Wheel struct {
    Circle
    Spokes int
}

В таком варианте типы Circle и Wheel имеют по одному анонимному полю. В этом случае говорят, что тип Point **встроен**(embedded) в тип Circle и тип Circle встроен в тип Wheel. Благодаря встраиванию мы можем обращаться к листовым именам воображаемого дерева без использования промежуточных звеньев:

In [18]:
var w Wheel
w.X = 0       // то же самое, что и w.Circle.Point.X = 0
w.Y = 9       // то же самое, что и w.Circle.Point.Y = 9
w.Radius = 10 // то же самое, что и w.Circle.Radius = 10
w.Spokes = 30

Явное обращение к полям, которое приведено в комментариях, при использовании встраивания продолжает работать. Поля  Point и Circle на самом деле имеют названия, которые совпадают с типом, но их можно пропускать.

К сожалению для литералов структур это **не работает**. Поэтому следующие объявления вызовут ошибку компиляции:  
w = Wheel{0, 9, 10, 30}  
w = Wheel{X: 8, Y: 8, Radius: 10, Spokes: 30}

Литерал должен в точности следовать форме объявления:

In [19]:
w = Wheel{Circle{Point{0, 9}, 10}, 30}

w = Wheel{
    Circle: Circle{
        Point: Point{X: 0, Y: 9},
        Radius: 10,
    },
    Spokes: 30, // запятая тут необходима также как и в Radius'е.
}

Т.к. анонимные поля имеют неявное имя, которое совпадает с названием типа, **нельзя создавать** два анонимных поля одинакового типа.  
Неявное имя также влияет и на экспортируемость. Если тип начинается с маленькой буквы, такое же имя будет и у неявного имени, следовательно поле не будет экспортировано. В нашем примере, если окружность была бы объявлена как circle, то выражение  
w.X = 8  
работало бы, но явная длинная форма обращения, которая приведена в комментариях в примере выше, за пределами пакета, в котором объявлется этот тип, не работала бы.